In [ ]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

from glob import glob
import os
from tqdm import tqdm
from functools import cmp_to_key
import json
import random
import math
import numpy as np
from datetime import datetime
from pytz import timezone
import shutil
from tensorflow import keras
import logging
import traceback
logging.basicConfig(level=logging.ERROR)

In [ ]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸"

from __future__ import print_function
from __future__ import absolute_import

######################################
# Import Library
######################################
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
import os
from pytz import timezone
import numpy as np
from utils.dev_env_info import getDevEnvInfo
from utils.data import GetOnOffDataset
from glob import glob
from sklearn.metrics import classification_report, confusion_matrix

######################################
# Set GPU
######################################
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


######################################
# Set GPU Memory
######################################
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
#   except RuntimeError as e:
#     print(e)


######################################
# Set Functions
######################################
def getConfusionMatrixLog(confusion_matrix_array, class_name):
    dst = ''
    dst += '{0:15s}{1:15s}{2:s}\n'.format('', '', 'Predicted Class')
    for idx in range(len(class_name) + 1):
        if idx == 0:
            dst += '{0:15s}{1:15s}'.format('', '')
            for name in class_name:
                dst += '{0:15s}'.format(name)

        else:
            if idx == 1:
                dst += '\n{0:15s}'.format('True Class')
                dst += '{0:15s}'.format(class_name[idx - 1])

            else:
                dst += '{0:15s}{1:15s}'.format('', class_name[idx - 1])

            for value in confusion_matrix_array[idx - 1]:
                dst += '{0:15s}'.format(str(value))
            dst += '\n'

    return dst


def getClassReportLog(class_report_dict, class_name):
    dst = ''
    dst += '{0:15s}{1:15s}{2:15s}{3:15s}\n'.format('', 'Precision', 'Recall', 'F1-score')
    for name in class_name:
        precision = class_report_dict[name]['precision']
        precision = '{0:.2f}'.format(precision)
        recall = class_report_dict[name]['recall']
        recall = '{0:.2f}'.format(recall)
        f1_score = class_report_dict[name]['f1-score']
        f1_score = '{0:.2f}'.format(f1_score)
        dst += '{0:15s}{1:15s}{2:15s}{3:15s}\n'.format(name,
                                                       precision,
                                                       recall,
                                                       f1_score)

    precision = class_report_dict['weighted avg']['precision']
    precision = '{0:.2f}'.format(precision)
    recall = class_report_dict['weighted avg']['recall']
    recall = '{0:.2f}'.format(recall)
    f1_score = class_report_dict['weighted avg']['f1-score']
    f1_score = '{0:.2f}'.format(f1_score)
    dst += '\n{0:15s}{1:15s}{2:15s}{3:15s}\n'.format('Total',
                                                     precision,
                                                     recall,
                                                     f1_score)

    return dst


######################################
# Global Parameters
######################################
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
cur_dir = os.getcwd()
dataset_dir = os.path.join(cur_dir, "dataset")
model_save_dir = os.path.join(cur_dir, '')
f = open("/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/output/evaluate_log.txt", 'w')
class_name = ['Get On', 'Get Off', 'Idle']


######################################
# Get Development Environment Info
######################################
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' 개발 환경 정보'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

tmp_str = getDevEnvInfo()
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)


In [ ]:
class GetOnOffData:                  #data.py에서 가져온 class
    def __init__(self, original_data_dir, get_on_off_data_dir, dataset_dir, train_val_rate):
        self.original_data_list = self.__getOriginalDataList(original_data_dir)
        self.original_data_dir = original_data_dir
        self.get_on_off_data_dir = get_on_off_data_dir
        self.train_val_rate = train_val_rate
        self.original_num = 0
        self.get_on_json_list = []
        self.get_off_json_list = []
        self.idle_json_list = []
        self.get_on_num = 0
        self.get_off_num = 0
        self.idle_num = 0
        self.dataset_dir = dataset_dir
        self.train_get_on_num = 0
        self.val_get_on_num = 0
        self.test_get_on_num = 0
        self.train_get_off_num = 0
        self.val_get_off_num = 0
        self.test_get_off_num = 0
        self.train_idle_num = 0
        self.val_idle_num = 0
        self.test_idle_num = 0
        self.train_num = 0
        self.val_num = 0
        self.test_num = 0
        self.train_list = []
        self.val_list = []
        self.test_list = []

   def makeGetOnOffData_getPrediction(self):
        # Get Model(AIhub에서 제공한 모델 원본)
        loaded_model = keras.models.load_model("/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/fs_lstm_get_on_off_chC.h5")

        # Make dir
        get_on_dir = os.path.join(self.get_on_off_data_dir, 'get_on')
        get_off_dir = os.path.join(self.get_on_off_data_dir, 'get_off')
        idle_dir = os.path.join(self.get_on_off_data_dir, 'idle')
        os.makedirs(get_on_dir, exist_ok=True)
        os.makedirs(get_off_dir, exist_ok=True)
        os.makedirs(idle_dir, exist_ok=True)

        # Count Parameters
        get_on_count = 0
        get_off_count = 0
        idle_count = 0

        # file name parameters
        get_on_data_name = 'get_on_'
        get_off_data_name = 'get_off_'
        idle_data_name = 'idle_'

        # Make get on off data
        for json_list in tqdm(self.original_data_list):
            # Get Passenger Data
            for i in range(len(json_list) - 2):
                # Get Json data
                json_path_1 = os.path.join(json_list[i])
                json_path_2 = os.path.join(json_list[i + 1])
                json_path_3 = os.path.join(json_list[i + 2])
                try:
                    with open(json_path_1, 'r') as json_file:
                        json_data_1 = json.load(json_file)
                except Exception:
                    raise Exception('json_path_1: ', json_path_1)
                try:
                    with open(json_path_2, 'r') as json_file:
                        json_data_2 = json.load(json_file)
                except Exception:
                    raise Exception('json_path_2: ', json_path_2)
                try:
                    with open(json_path_3, 'r') as json_file:
                        json_data_3 = json.load(json_file)
                except Exception:
                    raise Exception('json_path_3: ', json_path_3)

                # Get Img width & height
                width = json_data_1['info']['width']
                height = json_data_1['info']['height']

                # Get Valid data id and Append data
                if len(json_data_1['annotations']) != 0 and len(json_data_2['annotations']) != 0 and len(
                        json_data_3['annotations']) != 0:
                    for annotation_1 in json_data_1['annotations']:
                        id_1 = annotation_1['id']
                        for annotation_2 in json_data_2['annotations']:
                            id_2 = annotation_2['id']
                            if id_1 != id_2:
                                continue
                            else:
                                for annotation_3 in json_data_3['annotations']:
                                    id_3 = annotation_3['id']
                                    if id_2 != id_3:
                                        continue
                                    else:
                                        # Checking neck keypoint is exist
                                        if annotation_1['keypoints'][26] == 0 and annotation_2['keypoints'][26] == 0 and \
                                                annotation_3['keypoints'][26] == 0:
                                            break
                                        # Checking head keypoint is exist
                                        elif annotation_1['keypoints'][29] == 0 and annotation_2['keypoints'][
                                            29] == 0 and annotation_3['keypoints'][29] == 0:
                                            break
                                        # Checking right shoulder keypoint is exist
                                        elif annotation_1['keypoints'][38] == 0 and annotation_2['keypoints'][
                                            38] == 0 and annotation_3['keypoints'][38] == 0:
                                            break
                                        # Checking left shoulder keypoint is exist
                                        elif annotation_1['keypoints'][41] == 0 and annotation_2['keypoints'][
                                            41] == 0 and annotation_3['keypoints'][41] == 0:
                                            break

                                        # Get Data of neck keypoint
                                        neck_x_1 = annotation_1['keypoints'][24] / width
                                        neck_y_1 = annotation_1['keypoints'][25] / height
                                        neck_x_2 = annotation_2['keypoints'][24] / width
                                        neck_y_2 = annotation_2['keypoints'][25] / height
                                        neck_x_3 = annotation_3['keypoints'][24] / width
                                        neck_y_3 = annotation_3['keypoints'][25] / height

                                        # Get Data of head keypoint
                                        head_x_1 = annotation_1['keypoints'][27] / width
                                        head_y_1 = annotation_1['keypoints'][28] / height
                                        head_x_2 = annotation_2['keypoints'][27] / width
                                        head_y_2 = annotation_2['keypoints'][28] / height
                                        head_x_3 = annotation_3['keypoints'][27] / width
                                        head_y_3 = annotation_3['keypoints'][28] / height

                                        # Get Data of right shoulder keypoint
                                        r_shoulder_x_1 = annotation_1['keypoints'][36] / width
                                        r_shoulder_y_1 = annotation_1['keypoints'][37] / height
                                        r_shoulder_x_2 = annotation_2['keypoints'][36] / width
                                        r_shoulder_y_2 = annotation_2['keypoints'][37] / height
                                        r_shoulder_x_3 = annotation_3['keypoints'][36] / width
                                        r_shoulder_y_3 = annotation_3['keypoints'][37] / height

                                        # Get Data of left shoulder keypoint
                                        l_shoulder_x_1 = annotation_1['keypoints'][39] / width
                                        l_shoulder_y_1 = annotation_1['keypoints'][40] / height
                                        l_shoulder_x_2 = annotation_2['keypoints'][39] / width
                                        l_shoulder_y_2 = annotation_2['keypoints'][40] / height
                                        l_shoulder_x_3 = annotation_3['keypoints'][39] / width
                                        l_shoulder_y_3 = annotation_3['keypoints'][40] / height

                                        # Get Vector of neck->rShoulder
                                        vector_r_shoulder_x_1 = r_shoulder_x_1 - neck_x_1
                                        vector_r_shoulder_y_1 = r_shoulder_y_1 - neck_y_1
                                        vector_r_shoulder_x_2 = r_shoulder_x_2 - neck_x_2
                                        vector_r_shoulder_y_2 = r_shoulder_y_2 - neck_y_2
                                        vector_r_shoulder_x_3 = r_shoulder_x_3 - neck_x_3
                                        vector_r_shoulder_y_3 = r_shoulder_y_3 - neck_y_3

                                        # Get Vector of neck->lShoulder
                                        vector_l_shoulder_x_1 = l_shoulder_x_1 - neck_x_1
                                        vector_l_shoulder_y_1 = l_shoulder_y_1 - neck_y_1
                                        vector_l_shoulder_x_2 = l_shoulder_x_2 - neck_x_2
                                        vector_l_shoulder_y_2 = l_shoulder_y_2 - neck_y_2
                                        vector_l_shoulder_x_3 = l_shoulder_x_3 - neck_x_3
                                        vector_l_shoulder_y_3 = l_shoulder_y_3 - neck_y_3

                                        # Get Vector of neck->head
                                        vector_head_x_1 = head_x_1 - neck_x_1
                                        vector_head_y_1 = head_y_1 - neck_y_1
                                        vector_head_x_2 = head_x_2 - neck_x_2
                                        vector_head_y_2 = head_y_2 - neck_y_2
                                        vector_head_x_3 = head_x_3 - neck_x_3
                                        vector_head_y_3 = head_y_3 - neck_y_3

                                        # Push Data
                                        data = [[0 for col in range(8)] for row in range(3)]
                                        data[0] = [neck_x_1, neck_y_1,
                                                   vector_head_x_1, vector_head_y_1,
                                                   vector_r_shoulder_x_1, vector_r_shoulder_y_1,
                                                   vector_l_shoulder_x_1, vector_l_shoulder_y_1]
                                        data[1] = [neck_x_2, neck_y_2,
                                                   vector_head_x_2, vector_head_y_2,
                                                   vector_r_shoulder_x_2, vector_r_shoulder_y_2,
                                                   vector_l_shoulder_x_2, vector_l_shoulder_y_2]
                                        data[2] = [neck_x_3, neck_y_3,
                                                   vector_head_x_3, vector_head_y_3,
                                                   vector_r_shoulder_x_3, vector_r_shoulder_y_3,
                                                   vector_l_shoulder_x_3, vector_l_shoulder_y_3]

                                        data_dict = dict()
                                        data_dict['data'] = data
                                        data_dict['original_data'] = []
                                        for i in range(3):
                                            tmp_dict = dict()
                                            tmp_dict['time_step'] = i + 1
                                            if i == 0:
                                                tmp_dict['original_data_path'] = json_path_1
                                            elif i == 1:
                                                tmp_dict['original_data_path'] = json_path_2
                                            else:
                                                tmp_dict['original_data_path'] = json_path_3
                                            tmp_dict['id'] = annotation_3['id']

                                            data_dict['original_data'].append(tmp_dict)


                                        # Count num of get_on get_off idle
                                        if annotation_3['get_on']:
                                            get_on_count += 1
                                            data_dict['label'] = 0
                                            json_file_name = get_on_data_name + str(get_on_count) + '.json'
                                            json_file_path = os.path.join(get_on_dir, json_file_name)
                                            if not os.path.exists(json_file_path):
                                                with open(json_file_path, 'w') as f:
                                                    json.dump(data_dict, f, indent=4)

                                        elif annotation_3['get_off']:
                                            get_off_count += 1
                                            data_dict['label'] = 1
                                            json_file_name = get_off_data_name + str(get_off_count) + '.json'
                                            json_file_path = os.path.join(get_off_dir, json_file_name)
                                            if not os.path.exists(json_file_path):
                                                with open(json_file_path, 'w') as f:
                                                    json.dump(data_dict, f, indent=4)

                                        else:
                                            idle_count += 1
                                            data_dict['label'] = 2
                                            json_file_name = idle_data_name + str(idle_count) + '.json'
                                            json_file_path = os.path.join(idle_dir, json_file_name)
                                            if not os.path.exists(json_file_path):
                                                with open(json_file_path, 'w') as f:
                                                    json.dump(data_dict, f, indent=4)

                                        break

                                        # 의도 분석 모델의 예측 결과 얻기
                                        with open(json_file_path, 'r') as f:  # 1)생성한 input json 파일에서 모델 입력값 얻어오기
                                            json_data = json.load(f)
                                            
                                        input_data = json_data['data']
                                        prediction = loaded_model(input_data)   # 2)모델 예측 결과 얻기
                                        
                                        # 3)json파일의 사람객체에 의도분석 결과 할당하기
                                        if prediction = keras.utils.to_categorical(0, num_classes=3):
                                          with open(json_path_3, 'r') as file:
                                              temp_json = json.load(file)
                                              temp_json["annotations"][id_3]["get_on"] = True
                                              temp_json["annotations"][id_3]["get_off"] = False
                                          with open(json_path_3, 'w') as file:
                                              json.dump(temp_json, file, indent=4)

                                        elif prediction = keras.utils.to_categorical(1, num_classes=3):
                                          with open(json_path_3, 'r') as file:
                                              temp_json = json.load(file)
                                              temp_json["annotations"][id_3]["get_off"] = True
                                              temp_json["annotations"][id_3]["get_on"] = False
                                          with open(json_path_3, 'w') as file:
                                              json.dump(temp_json, file, indent=4)
                                        
                                        else:
                                          with open(json_path_3, 'r') as file:
                                              temp_json = json.load(file)
                                              temp_json["annotations"][id_3]["get_on"] = False
                                              temp_json["annotations"][id_3]["get_off"] = False
                                          with open(json_path_3, 'w') as file:
                                              json.dump(temp_json, file, indent=4)

                            break

        self.get_on_num = get_on_count
        self.get_off_num = get_off_count
        self.idle_num = idle_count

In [ ]:
######################################
# Get Test Data
######################################
# Start make test numpy array
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Test 데이터 정보 가져오기 시작'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

get_on_off_dataset = GetOnOffDataset(dataset_dir)
test_list = get_on_off_dataset.test_list

# Get test data list
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Test 데이터 목록'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

tmp_str = ''
for idx, test_json_path in enumerate(test_list):
    tmp_str += str(idx + 1) + '. ' + test_json_path + '\n'
    print(test_json_path)
tmp_str += '\n'
f.write(tmp_str)

# Get test numpy array
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Test 데이터 numpy 배열 정보'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)
test_x, test_y = get_on_off_dataset.getTestDatasetNumpyArray()
log_str = ''
log_str += 'Test_X Shape: {}\n'.format(test_x.shape)
log_str += 'Test_Y Shape: {}\n'.format(test_y.shape)
log_str += '\n'
f.write(log_str)
print(log_str)

# Get Test Dataset API
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_dataset = test_dataset.batch(1)

######################################
# Start Evaluate
######################################
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' 모델 평가 시작'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

y_true = []
y_pred = []
for idx, (input_x, input_y) in enumerate(test_dataset.as_numpy_iterator()):
    label = np.argmax(input_y, axis=1)
    prediction = np.argmax(model(input_x, training=False), axis=1)

    cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
    timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
    tmp_str = timestamp_str + ' ' + str(idx+1) + '. ' + test_list[idx] + ' Test 데이터 인풋 결과'
    log_str = tmp_str + '\n'
    f.write(log_str)
    print(tmp_str)

    tmp_str = ''
    label_name = ''
    pred_name = ''

    if label == 0:
        label_name = 'Get On'
    elif label == 1:
        label_name = 'Get Off'
    else:
        label_name = 'Idle'

    if prediction == 0:
        pred_name = 'Get On'
    elif prediction == 1:
        pred_name = 'Get Off'
    else:
        pred_name = 'Idle'

    tmp_str += '정답: {}\n'.format(label_name)
    tmp_str += '예측: {}\n'.format(pred_name)
    f.write(tmp_str)
    print(tmp_str)

    y_true += list(label)
    y_pred += list(prediction)